In [1]:
import torch
import pandas as pd
import json

/home/fred/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1371886/3064727310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [16]:
# LOAD TENSOR DATA
data = 'valid'
train_data = pd.read_csv(f'./data/WN18RR_torch/raw/{data}.txt', sep='\t',header=None)
train_data.columns = ['subject', 'relation','object']

In [17]:
train_data

,subject,relation,object
0,2174461,_hypernym,2176268
1,5074057,_derivationally_related_form,2310895
2,8390511,_synset_domain_topic_of,8199025
3,2045024,_member_meronym,2046321
4,4758181,_hypernym,4757864
...,...,...,...
3029,11772154,_member_meronym,11772408
3030,102927,_derivationally_related_form,2226172
3031,2194138,_derivationally_related_form,5715864
3032,4842029,_hypernym,4841810


In [18]:
unique_values = train_data['relation'].unique()
mapping = {value: i for i,value in enumerate(unique_values)}
print(mapping)
train_data['relation'] = train_data['relation'].map(mapping)
with open('data/WN18RR_torch/sequences/mapping','w') as outfile:
    json.dump(mapping,outfile,indent=4)



{'_hypernym': 0, '_derivationally_related_form': 1, '_synset_domain_topic_of': 2, '_member_meronym': 3, '_instance_hypernym': 4, '_has_part': 5, '_verb_group': 6, '_also_see': 7, '_member_of_domain_usage': 8, '_member_of_domain_region': 9, '_similar_to': 10}


In [19]:
train_data.insert(loc=0,column='column0',value=['[CLS]']*len(train_data))
train_data.insert(loc=2,column='column2',value=['[SEP1]']*len(train_data))
train_data.insert(loc=4,column='column4',value=['[SEP2]']*len(train_data))
train_data.insert(loc=6,column='column6',value=['[END]']*len(train_data))

In [20]:
# train_data = train_data.drop('column0',axis=1)
# train_data = train_data.drop('column2',axis=1)
# train_data = train_data.drop('column4',axis=1)
# train_data = train_data.drop('column6',axis=1)

In [21]:
train_data

,column0,subject,column2,relation,column4,object,column6
0,[CLS],2174461,[SEP1],0,[SEP2],2176268,[END]
1,[CLS],5074057,[SEP1],1,[SEP2],2310895,[END]
2,[CLS],8390511,[SEP1],2,[SEP2],8199025,[END]
3,[CLS],2045024,[SEP1],3,[SEP2],2046321,[END]
4,[CLS],4758181,[SEP1],0,[SEP2],4757864,[END]
...,...,...,...,...,...,...,...
3029,[CLS],11772154,[SEP1],3,[SEP2],11772408,[END]
3030,[CLS],102927,[SEP1],1,[SEP2],2226172,[END]
3031,[CLS],2194138,[SEP1],1,[SEP2],5715864,[END]
3032,[CLS],4842029,[SEP1],0,[SEP2],4841810,[END]


In [22]:
train_data_subject_masked = train_data.copy(deep=True)
masked_subject = train_data_subject_masked.pop('subject')
train_data_object_masked = train_data.copy(deep=True)
masked_object = train_data_object_masked.pop('object')

In [23]:
train_data_subject_masked.insert(loc=1,column='subject',value=['[MASK]']*len(train_data_subject_masked))
train_data_object_masked.insert(loc=5,column='object',value=['[MASK]']*len(train_data_object_masked))

In [24]:
train_data_object_masked

,column0,subject,column2,relation,column4,object,column6
0,[CLS],2174461,[SEP1],0,[SEP2],[MASK],[END]
1,[CLS],5074057,[SEP1],1,[SEP2],[MASK],[END]
2,[CLS],8390511,[SEP1],2,[SEP2],[MASK],[END]
3,[CLS],2045024,[SEP1],3,[SEP2],[MASK],[END]
4,[CLS],4758181,[SEP1],0,[SEP2],[MASK],[END]
...,...,...,...,...,...,...,...
3029,[CLS],11772154,[SEP1],3,[SEP2],[MASK],[END]
3030,[CLS],102927,[SEP1],1,[SEP2],[MASK],[END]
3031,[CLS],2194138,[SEP1],1,[SEP2],[MASK],[END]
3032,[CLS],4842029,[SEP1],0,[SEP2],[MASK],[END]


In [25]:
def df_to_text(row):
    return ' '.join(row.astype(str).values)

textlines = train_data.apply(df_to_text,axis=1).tolist()
textlines_subject_masked_input = train_data_subject_masked.apply(df_to_text,axis=1).tolist()
textlines_subject_masked_output = masked_subject.astype(str).tolist()
textlines_object_masked_input = train_data_object_masked.apply(df_to_text,axis=1).tolist()
textlines_object_masked_output = masked_object.astype(str).tolist()

output_text = '\n'.join(textlines)
output_text_subject_masked_input = '\n'.join(textlines_subject_masked_input)
output_text_subject_masked_ouput = '\n'.join(textlines_subject_masked_output)
output_text_object_masked_input = '\n'.join(textlines_object_masked_input)
output_text_object_masked_ouput = '\n'.join(textlines_object_masked_output)

In [26]:

with open (f'data/WN18RR_torch/sequences/{data}_seq.txt','w') as f:
    f.write(output_text)
with open (f'data/WN18RR_torch/sequences/{data}_seq_subject_masked_input.txt','w') as f:
    f.write(output_text_subject_masked_input)
with open (f'data/WN18RR_torch/sequences/{data}_seq_subject_masked_ouput.txt','w') as f:
    f.write(output_text_subject_masked_ouput)
with open (f'data/WN18RR_torch/sequences/{data}_seq_object_masked_input.txt','w') as f:
    f.write(output_text_object_masked_input)
with open (f'data/WN18RR_torch/sequences/{data}_seq_object_masked_ouput.txt','w') as f:
    f.write(output_text_object_masked_ouput)

In [9]:
#[CLS] 00260881 [SEP1] 3 [SEP2] 10211 [END]